In [173]:
import pandas as pd
df_matches = pd.read_csv("matches.csv")
df_matches.head(3)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN


In [174]:
df_deliveries = pd.read_csv("deliveries.csv")
df_deliveries.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN


In [175]:
print(df_matches.shape)
print(df_deliveries.shape)

(756, 18)
(179078, 21)


In [176]:
#plotting every team wins not only RCB and KKR. 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
df_winner_matches = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner_matches.columns = ['winner','total_wins']
df_winner_matches = df_winner_matches.sort_values('total_wins',ascending=False)

data = go.Bar(x = df_winner_matches['winner'],y = df_winner_matches['total_wins'])
layout = go.Layout(title = 'Total wins by every team in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

In [177]:
print(df_matches.columns)
print(df_deliveries.columns)

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')
Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


In matches data frame columns like umpire1,umpire2,umpire3 are not useful because these days they are using technology. So we cant decide which umpire is more useful for predicting anything related to match.

Date is also not useful because we already have season column which is also almost equal to date.

City and venue also not important features for this year IPL. Because IPL2020 is happening in UAE, only 2014 ipl was happened in UAE that to half season.

In [178]:
df_matches = df_matches.drop(['city','venue','date','umpire1','umpire2','umpire3'], axis = 1) 
df_matches.head(2)

,id,season,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
0,1,2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh
1,2,2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith


In [179]:
print(df_matches["team1"].value_counts())

Mumbai Indians                 101
Kings XI Punjab                 91
Chennai Super Kings             89
Royal Challengers Bangalore     85
Kolkata Knight Riders           83
Delhi Daredevils                72
Rajasthan Royals                67
Sunrisers Hyderabad             63
Deccan Chargers                 43
Pune Warriors                   20
Gujarat Lions                   14
Rising Pune Supergiant           8
Rising Pune Supergiants          7
Kochi Tuskers Kerala             7
Delhi Capitals                   6
Name: team1, dtype: int64


In [180]:
#In 2019 delhi daredevils name changed to delhi capitals. So i am replacing old name with new name for all the rows.
df_matches = df_matches.replace(to_replace ="Delhi Daredevils", value = "Delhi Capitals") 

df_deliveries = df_deliveries.replace(to_replace = "Delhi Daredevils", value = "Delhi Capitals")

In [181]:
# We are predicting results of the match between RCB and KKR. So i am taking only rows which has delhi or mumabi from team1 and team2 columns.
df_matches = df_matches[(df_matches['team1'] == 'Royal Challengers Bangalore') | (df_matches['team1'] == 'Kolkata Knight Riders') | 
                        (df_matches['team2'] == 'Royal Challengers Bangalore') | (df_matches['team2'] == 'Kolkata Knight Riders')]

# df_deliveries = df_deliveries[(df_deliveries['batting_team'] == 'Royal Challengers Bangalore') | (df_deliveries['batting_team'] == 'Kolkata Knight Riders') |
#                               (df_deliveries['batting_team'] == 'Royal Challengers Bangalore') | (df_deliveries['batting_team'] == 'Kolkata Knight Riders')]
df_matches.head(2)

,id,season,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
0,1,2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh
2,3,2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn


In [182]:
#checking null values in matches dataset
print(df_matches.isnull().sum())

id                 0
season             0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             3
win_by_runs        0
win_by_wickets     0
player_of_match    3
dtype: int64


In [183]:
#printing null rows in matches dataset
df1_null = df_matches[df_matches.isna().any(axis=1)]
df1_null.head()

,id,season,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
545,546,2015,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,no result,0,NaN,0,0,NaN
570,571,2015,Delhi Capitals,Royal Challengers Bangalore,Royal Challengers Bangalore,field,no result,0,NaN,0,0,NaN
744,11340,2019,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,no result,0,NaN,0,0,NaN


In [184]:
#After seeing above table we can clearly understood that 3 matches are not played beacuse of rain, badlight or pitch condition. 
#So i am replacing those null values in winner column with no result and palyer of the match column null values with not declared.
df_matches["winner"].fillna("No result", inplace = True)
df_matches['player_of_match'].fillna("Not declared", inplace = True)
print(df_matches.shape)

(334, 12)


In [185]:
#checking null values in deliveries dataset
print(df_deliveries.isnull().sum())

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
is_super_over            0
wide_runs                0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
player_dismissed    170244
dismissal_kind      170244
fielder             172630
dtype: int64


Player_dismissed, dismissal_kind and fielder are having null values. But if you consider today questions only player_dismissed is useful for calculating average of batsman. So i am removing remaining two features.

Filling player_dismissed null values with no wicket

In [186]:
#dropping columns
df_deliveries = df_deliveries.drop(['dismissal_kind','fielder'], axis = 1) 
#filling null values
df_deliveries["player_dismissed"].fillna("No Wicket", inplace = True)
df_deliveries.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket


## Exploratory Data Analysis(EDA)

In [187]:
#plotting KKR and RCB wins  
import plotly.graph_objs as go
df_winner = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)[:2]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Total wins by every team in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()


In [188]:
df_total_wins = df_matches[['id','winner']].groupby('winner').size().reset_index().sort_values(0,ascending = False)[:2]
df_total_wins.columns = ['Team','Total_wins']

df_total_wins

,Team,Total_wins
6,Kolkata Knight Riders,92
12,Royal Challengers Bangalore,84


By looking at above observations it is clearly indicating that KKR won more matches than RCB in IPL till 2019. But it is not that much difference just 8 matches. So we cant predict match result by using with this data.

In [189]:
#Here i am checking how many times toss winning team won the match

import numpy as np
print("Number of matches won by toss winning team : " , df_matches[df_matches.toss_winner == df_matches.winner].shape[0])


Number of matches won by toss winning team :  178


So out 334 matches played by KKR and RCB 178 times toss winning team won the match. So here also almost 50-50% winning ratio between toss winning team and loosing team.

For first question i am predicting who win the toss that team won the match.

In [190]:
df_matches['toss_decision'].value_counts()

field    220
bat      114
Name: toss_decision, dtype: int64

Here toss winning captain trying to field first most of the time. But in this ipl batting first team won more matches. So my prediction is completely opposite to above data.

In [191]:
#Here i am tking all 5 RCB bowlers who are bowling in today match against dinesh karthik. 
#I think except Isuru udana and Shivam dube every RCB bowler bowled atlease one ball against Dinesh karthik
#Note : i am considering all the matches data not only KKR matches for dinesh karthik question.
# Because DK played for diffrent franchises(I think 5teams) in his IPL carrer.
df_Karthik_all = df_deliveries[(df_deliveries['batsman'] == 'KD Karthik') & ((df_deliveries['bowler'] == 'YS Chahal')|
                                                                             (df_deliveries['bowler'] == 'CH Morris') |
                                                                             (df_deliveries['bowler'] == 'Washington Sundar	') |
                                                                             (df_deliveries['bowler'] == 'N Saini') |
                                                                             (df_deliveries['bowler'] == 'I Udana'))]
print("Dinesh Karthik runs against RCB bowlers : " , df_Karthik_all['batsman_runs'].sum())
print(" In Number of matches Dinesh karthik got batting aginst RCB bowlers : ",df_Karthik_all['match_id'].nunique())
print(" In Number of matches Dinesh karthik out against RCB bowlers : ",len(df_Karthik_all[df_Karthik_all['player_dismissed'] == 'KD Karthik']))

Dinesh Karthik runs against RCB bowlers :  73
 In Number of matches Dinesh karthik got batting aginst RCB bowlers :  12
 In Number of matches Dinesh karthik out against RCB bowlers :  4


In [192]:
#By looking at above observations Dinesh karthik is remained not out on 8 matches out of 12
#Now i am claculating DK average against RCB bowlers

Karthin_average = df_Karthik_all['batsman_runs'].sum()/len(df_Karthik_all[df_Karthik_all['player_dismissed'] == 'KD Karthik'])
print("Dinesh karthik average against RCB bowlers : ",Karthin_average)

Dinesh karthik average against RCB bowlers :  18.25


So in this match i am predicting Karthik score in between 11-20

In [193]:
#For no ball question i am filtering out the bowlers in both the team and checking how many no balls they bowled.
#Here i am not picking varun chakravarthy,shivm mavi and nagarkoti. I dont think they played ipl before this season except 1 or 2 matches.
df_KKR_RCB_bowlers = df_deliveries[((df_deliveries['bowler'] == 'YS Chahal') | (df_deliveries['bowler'] == 'CH Morris') |
                               (df_deliveries['bowler'] == 'Washington Sundar	') | (df_deliveries['bowler'] == 'N Saini') |
                               (df_deliveries['bowler'] == 'I Udana') | (df_deliveries['bowler'] == 'SP Narine') |
                               (df_deliveries['bowler'] == 'P Krishna') | (df_deliveries['bowler'] == 'AD Russell') |
                               (df_deliveries['bowler'] == 'PJ Cummins'))]
print("Number of no balls bowled by RCB and KKR bowlers : ",df_KKR_RCB_bowlers['noball_runs'].sum())
print("Number of balls bowled by RCB and KKR bowlers : ", df_KKR_RCB_bowlers.shape[0])

Number of no balls bowled by RCB and KKR bowlers :  27
Number of balls bowled by RCB and KKR bowlers :  7964


As per above observations out of 7585 bowls RCB and KKR bowlers bowled only 26 no balls. As for my cricket knowledge it is very less number of no balls. So i am predicting in today match they will conceed only 1 no ball.

In [194]:
#For number of fours scored by virat kohli question, first i am filtering out virat kohli from batsman list and 
# Number of fours from batsman_runs feature

df_Kohli_fours = df_deliveries[(df_deliveries['batsman'] == 'V Kohli') & (df_deliveries['batsman_runs'] == 4)]
df_Kohli_fours.shape[0]
df_Kohli_fours['match_id'].nunique()
print("Virat kohli Scored ", + df_Kohli_fours.shape[0]  , "fours in ", +  df_Kohli_fours['match_id'].nunique() , "matches")

Virat kohli Scored  482 fours in  135 matches


In [195]:
#Calculating number of fours per match
virat_fours_per_match = df_Kohli_fours.shape[0] / df_Kohli_fours['match_id'].nunique()
print(virat_fours_per_match)

3.5703703703703704


Virat Scored almost 4 fours per match as per above observation. 
For this question i am not taking virat against individual bowler in KKR because all KKR bowlers are new to IPL and they didnt played enough IPL except Sunil narine and russell.

So i am predicting virat will hit 4 fours in today match.